In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import pandas as pd

In [2]:
URAdf=pd.read_csv("../Mid terms/Past transactions (condo and apartments).csv")
URAdf.head()
properties=URAdf.loc[URAdf["Tenure"]!="Freehold"]["Project Name"].unique()
print(properties)

['NEPTUNE COURT' 'VILLA MARINA' 'PEBBLE BAY' 'WATER PLACE'
 'TANJONG RIA CONDOMINIUM' 'MANDARIN GARDENS' 'LEGENDA AT JOO CHIAT'
 'LAGUNA PARK' 'THE SHORE RESIDENCES' 'CASUARINA COVE' 'LAGOON VIEW'
 'SANCTUARY GREEN' 'SEASIDE RESIDENCES' 'RIVEREDGE' "COTE D'AZUR"
 'SILVERSEA' 'COSTA RHU' 'CAMELOT BY-THE-WATER' 'THE RED HOUSE'
 'DUNMAN VIEW' 'EAST BAY GARDENS' 'RESIDENTIAL APARTMENTS' 'ROXY SQUARE'
 'FRANKEL ESTATE' 'SEASIDE PARK' 'KATONG PARK TOWERS']


In [3]:
"""
condonames=['VILLA MARINA','PEBBLE BAY','WATER PLACE','TANJONG RIA CONDOMINIUM',
 'MANDARIN GARDENS','LEGENDA AT JOO CHIAT','THE SHORE RESIDENCES',
 'CASUARINA COVE','SANCTUARY GREEN','RIVEREDGE',"COTE DAZUR",'SILVERSEA',
 'COSTA RHU','CAMELOT BY-THE-WATER','DUNMAN VIEW','KATONG PARK TOWERS']
"""
"""
apartnames=['NEPTUNE COURT','LAGUNA PARK','LAGOON VIEW','SEASIDE RESIDENCES',
'THE RED HOUSE','EAST BAY GARDENS','RESIDENTIAL APARTMENTS','ROXY SQUARE','FRANKEL ESTATE','SEASIDE PARK']
"""

for i in range(len(properties)):
    properties[i]=properties[i].lower().replace(" ","-")

In [4]:
#Xpaths 
hamburger_btn = "//div[@class='jsx-3208738789 header-menu-item']"
login_btn = "//div[@class='jsx-1385425452 menu-item']"
agent_btn = "//span[text()='Agent']"
login_btn2 = "//center[text()='Login']"
login_btn3 = "//center[text()='LOGIN']"

attr_tbl = '//*[@id="summary"]/div/div[2]/div/div[1]/span/div/div/div/div/div/div/table/tbody'

showmore='//*[@id="summary"]/div/div[2]/div/div[1]/span/div/div/div/div/div/div/table/tbody/tr[6]/td[3]/h4/div/span/span[1]/span/span/span[2]/a'

transaction_tbl='//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/ul/li[8]/div/div[1]'
transactions_tab = '//*[@id="SalesTransaction-tab-sales"]'
profitable_tab = '//*[@id="SalesTransaction-tab-profitable"]'
unprofitable_tab = '//*[@id="SalesTransaction-tab-unprofitable"]'

ddmenu='//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/ul/li[8]/div/div[1]'
rows100='//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/ul/li[8]/div/div[2]/div/div/div/div[2]/div/div/div/div[4]'

annoyingpopout='//*[@class="_hj-kWRoL__styles__openStateToggle"]'

In [5]:
def clickbutton(path):
    WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.XPATH, path)))
    element = driver.find_element(By.XPATH, path)
    element.click()

def readrow(prop,i):
    row = driver.find_element(By.XPATH,'//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/div/div/div/table/tbody/tr[' + str(i) + ']')
    row = row.text.splitlines()

    #splitting strata and private
    typestrata = row[7].split()
    row[7] = typestrata[0]
    row.insert(8 , typestrata[1])
    row.insert(0 , prop)
    return row

def readtext(xpath): 
    text=driver.find_element(By.XPATH,xpath).text
    return text

In [6]:
#scraping condo basic attributes
def CondoAtttributes():
    
    try:
        #click on See more
        element = driver.find_element(By.XPATH, showmore)
        element.click()
    except:
        print("no show more")
    finally:
        #scraping from table
        basicinfodict={}
        rows= len( readtext(attr_tbl).splitlines() ) /3
        
        for i in range (1,int(rows)):
            colname = driver.find_element(By.XPATH,'//*[@id="summary"]/div/div[2]/div/div[1]/span/div/div/div/div/div/div/table/tbody/tr['+str(i)+']/td[1]/h3').text
            detail = driver.find_element(By.XPATH,'//*[@id="summary"]/div/div[2]/div/div[1]/span/div/div/div/div/div/div/table/tbody/tr['+str(i)+']/td[3]/h4').text
            basicinfodict[colname]=detail

        
        PostalCode      =   basicinfodict["Postal Code"].splitlines()
        ProjectName     =   basicinfodict["Project Name"]
        District        =   basicinfodict['District/Planning Area'].split(" / ")[0]
        NumberofUnits   =   basicinfodict['Number of Units'].split(" ")[0]
        PriceRange      =   basicinfodict['Indicative Price Range / Average*'].split()[-2]
        Tenure          =   basicinfodict['Tenure']
        #TenureFrom      =   int(Tenure[-4:])
        #Age             =   2023-TenureFrom

        Records         =   readtext(transactions_tab)
        Records         =   int("".join(re.findall("\d",Records)))

        Profitable      =   readtext(profitable_tab)
        Profitable      =   int("".join(re.findall("\d",Profitable)))

        Unprofitable    =   readtext(unprofitable_tab)
        Unprofitable    =   int("".join(re.findall("\d",Unprofitable)))

        basicinfolist = [PostalCode, ProjectName,District,NumberofUnits,PriceRange,Tenure,Records,Profitable,Unprofitable]
        
    return basicinfolist

In [7]:
#start chrome
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless')
path="../Dependencies/chromedriver.exe"
driver = webdriver.Chrome(executable_path=path, options=options)

#open website
driver.get("https://www.edgeprop.sg/")

#Click on hamburger icon to open menu
clickbutton(hamburger_btn)

#click on login button to open menu
clickbutton(login_btn)

#click agent login button
clickbutton(agent_btn)

#input mobile number
number = "number"  #input own account
password = "password"
mobile_input = driver.find_element(By.ID, "username")
mobile_input.send_keys(number)
password_input = driver.find_element(By.ID, "password")
password_input.send_keys(password)

#login button
clickbutton(login_btn2)

C:\Users\melvc\AppData\Local\Temp/ipykernel_28800/2159069532.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


Click on login button if prompt for you'll be signed out elsewhere pops out

In [8]:
try:
    login_btn3 = WebDriverWait(driver,20).until(
                            EC.presence_of_element_located((By.XPATH, login_btn3))
                        )
    login_btn3.click()
    print("Login after prompted to sign out everywhere else")
except:
    print("No prompt to sign out everywhere else")

No prompt to sign out everywhere else


Extracting tables from individual condo page

In [9]:
#closing annoying pop-up
    #popup = WebDriverWait(driver,60).until(
    #    EC.presence_of_element_located((By.XPATH, annoyingpopout))
    #)
    #popup.click()
    #print("Annoying pop-up closed")

In [10]:
### Initialise Databases
#creating condo DF
condo_columns = ["Postal Code", 'Project Name', 'District', 'Number of Units', 'Indicative Price Range', 'Tenure', '# Records', '# Profitable sales', '# Unprofitable sales']
condosdf=pd.DataFrame(columns=condo_columns)

#creating transations DF
trans_columns = ['Project Name','Date', 'Area(sqft)', 'Bedrooms', 'Price(S$ psf)', 'Price (S$)', 'Type of Sale', 'Address', 'Type of Area', 'Purchaser Address', 'Source']
transactionsdf=pd.DataFrame(columns=trans_columns)

In [11]:
def generate_webscrape_df(properties):
    failed_condos_ls = []   # list to store which condos script failed to scrape data

    for prop in properties:
        temptransdf=pd.DataFrame(columns=trans_columns)

        try:
            driver.get("https://www.edgeprop.sg/condo-apartment/" + prop)

            #loading of webpage
            WebDriverWait(driver,60).until(
                EC.presence_of_element_located((By.XPATH,'//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/div/div/div/table/thead'))
            )
            print(prop +" loaded")

            #loading of table
            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH, transaction_tbl)))
            #print("table loaded")

            driver.implicitly_wait(10)

            #scraping condo attributes
            print("extracting attr")
            tempattrdf = CondoAtttributes()
            print(tempattrdf)

            recno = tempattrdf[-3]
            pages = recno // 10 + 1

            i=1 #page counter
            while i <= pages:
                print("Page", i)

                if i < pages: # for multi pages
                    for j in range(2, 12):
                        temptransdf.loc[len(temptransdf)] = readrow(prop,j)
                    #click next table
                    pageselector = driver.find_element(By.XPATH,'//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/ul')
                    next_btn = pageselector.find_element(By.CLASS_NAME,"ant-pagination-next")

                    #checking for annoying popout
                    try:
                        next_btn.click()
                    except:
                        print("Interupted by annoying pop-up")
                        popup = WebDriverWait(driver,60).until(
                            EC.presence_of_element_located((By.XPATH, annoyingpopout))
                        )
                        clickbutton(annoyingpopout)
                        print("Annoying pop-up closed")
                        next_btn.click()
                    i+=1

                else: #last page <10
                    numberofrec = recno % 10
                    #scraping of rows, loop through rows
                    for j in range (2 , numberofrec+2):
                        temptransdf.loc[len(temptransdf)] = readrow(prop,j)
                    i+=1 
                
            #add to main database
            print("concating")
            condosdf.loc[len(condosdf)] = tempattrdf
            print("concat attr")
            transactionsdf = pd.concat([transactionsdf, temptransdf])
            print("concat trans")
            print(f"{prop} successfully scraped")
        except:
            failed_condos_ls.append(prop)
            print(f"{prop} failed")
            
    return failed_condos_ls

First run, loop through all condos and store condos that failed to be scraped as output

In [12]:
failed_condos_ls = generate_webscrape_df(properties)

while len(failed_condos_ls)!=0:
    failed_condos_ls = generate_webscrape_df(failed_condos_ls)

neptune-court loaded
extracting attr
[['449025, 449026, 449027, 449028, 449029, 449030, 449031, 449032, 449033, 449034', 'Show less'], 'NEPTUNE COURT', 'D15', '751', '1,006', '99 Yrs From 01/09/1976, 99 Yrs From 01/11/1975', 649, 147, 23]
Page 1
Interupted by annoying pop-up
Annoying pop-up closed
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43
Page 44
Page 45
Page 46
Page 47
Page 48
Page 49
Page 50
Page 51
Page 52
Page 53
Page 54
Page 55
Page 56
Page 57
Page 58
Page 59
Page 60
Page 61
Page 62
Page 63
Page 64
Page 65
concating
concat attr
neptune-court failed
villa-marina loaded
extracting attr
[['457398, 457399, 457400, 457401, 457402, 457403, 457404, 457405, 457406', 'Show less'], 'VILLA MARINA', 'D15', '432'

Working to stop annoying pop-up from inteferring with results

In [1]:
pageselector = driver.find_element(By.XPATH,'//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/ul')
next_btn = pageselector.find_element(By.CLASS_NAME,"ant-pagination-next")
try:
    next_btn.click()
except:
    print("Interupted by annoying pop-up")
    popup = WebDriverWait(driver,60).until(
        EC.presence_of_element_located((By.XPATH, annoyingpopout))
    )
    popup.click()
    print("Annoying pop-up closed")
    next_btn.click()

NameError: name 'driver' is not defined

For remaining failed condos

In [16]:
temptransdf=pd.DataFrame(columns=trans_columns)

prop="neptune-court"
driver.get("https://www.edgeprop.sg/condo-apartment/" + prop)

#loading of webpage
WebDriverWait(driver,60).until(
    EC.presence_of_element_located((By.XPATH,'//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/div/div/div/table/thead'))
)
print(prop +" loaded")

#loading of table
WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH, transaction_tbl)))
#print("table loaded")

driver.implicitly_wait(10)

#scraping condo attributes
print("extracting attr")
tempattrdf = CondoAtttributes()
print(tempattrdf)

recno = tempattrdf[-3]
pages = recno // 10 + 1

i=1 #page counter
while i <= pages:
    print("Page", i)

    if i < pages: # for multi pages
        for j in range(2, 12):
            temptransdf.loc[len(temptransdf)] = readrow(prop,j)
        #click next table
        pageselector = driver.find_element(By.XPATH,'//*[@id="SalesTransaction-panel-sales"]/div/div/div/div/div/div[1]/div/div/div/ul')
        next_btn = pageselector.find_element(By.CLASS_NAME,"ant-pagination-next")

        #checking for annoying popout
        try:
            next_btn.click()
        except:
            print("Interupted by annoying pop-up")
            popup = WebDriverWait(driver,60).until(
                EC.presence_of_element_located((By.XPATH, annoyingpopout))
            )
            clickbutton(annoyingpopout)
            print("Annoying pop-up closed")
            next_btn.click()
        i+=1

    else: #last page <10
        numberofrec = recno % 10
        #scraping of rows, loop through rows
        for j in range (2 , numberofrec+2):
            temptransdf.loc[len(temptransdf)] = readrow(prop,j)
        i+=1 
    
#add to main database
print("concating")
condosdf.loc[len(condosdf)] = tempattrdf
print("concat attr")
transactionsdf = pd.concat([transactionsdf, temptransdf])
print("concat trans")
print(f"{prop} successfully scraped")

neptune-court loaded
extracting attr
[['449025, 449026, 449027, 449028, 449029, 449030, 449031, 449032, 449033, 449034', 'Show less'], 'NEPTUNE COURT', 'D15', '751', '1,006', '99 Yrs From 01/09/1976, 99 Yrs From 01/11/1975', 649, 147, 23]
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43
Page 44
Page 45
Page 46
Page 47
Page 48
Page 49
Page 50
Page 51
Page 52
Page 53
Page 54
Page 55
Page 56
Page 57
Page 58
Page 59
Page 60
Page 61
Page 62
Page 63
Page 64
Page 65
concating
concat attr
concat trans
neptune-court successfully scraped


In [1]:
temptransdf

NameError: name 'temptransdf' is not defined

In [18]:
transactionsdf

,Project Name,Date,Area(sqft),Bedrooms,Price(S$ psf),Price (S$),Type of Sale,Address,Type of Area,Purchaser Address,Source
0,pebble-bay,24 FEB 2023,850,1,"1,658","1,410,000",Resale,132 Tanjong Rhu Road #10-03,Strata,Private,URA
1,pebble-bay,2 FEB 2023,"1,894",3,"1,821","3,450,000",Resale,130 Tanjong Rhu Road #09-03,Strata,Private,URA
2,pebble-bay,12 DEC 2022,"2,174",3,"1,711","3,720,000",Resale,130 TANJONG RHU ROAD #02-05,Strata,Private,URA
3,pebble-bay,8 DEC 2022,"2,336",3,"1,670","3,900,000",Resale,132 TANJONG RHU ROAD #05-09,Strata,Private,URA
4,pebble-bay,17 NOV 2022,"2,336",3,"1,550","3,620,000",Resale,130 Tanjong Rhu Road #08-10,Strata,Private,URA
...,...,...,...,...,...,...,...,...,...,...,...
644,neptune-court,17 APR 1996,"1,270",3,484,"615,000",Resale,6 Marine Vista #18-19,Strata,HDB,URA
645,neptune-court,15 APR 1996,"1,270",3,512,"650,000",Resale,9 Marine Vista #23-03,Strata,HDB,URA
646,neptune-court,30 JAN 1996,"1,636",3,405,"662,000",Resale,3 Marine Vista #05-63,Strata,Private,URA
647,neptune-court,2 JAN 1996,"1,636",3,458,"750,000",Resale,4 Marine Vista #21-55,Strata,Private,URA


EXPORT TO CSV

In [ ]:
condosdfcondosdf.to_csv("../Raw Data/Edgeprop/condos_attr.csv", index=False)
transactionsdf.to_csv("../Raw Data/Edgeprop/condos_attr.csv", index=False)